# Imports

In [1]:
import os
import geopandas
import logging
import folium
import pandas as pd
import io
from PIL import Image

try:
    from PT3S import dxAndMxHelperFcts
except:
    import dxAndMxHelperFcts

In [2]:
pt3s_path = os.path.dirname(os.path.abspath(dxAndMxHelperFcts.__file__))

# Logging

In [3]:
logger = logging.getLogger()  

logFileName= r"Example1.log" 

loglevel = logging.DEBUG
logging.basicConfig(filename=logFileName
                        ,filemode='w'
                        ,level=loglevel
                        ,format="%(asctime)s ; %(name)-60s ; %(levelname)-7s ; %(message)s")    

fileHandler = logging.FileHandler(logFileName)     

logger.addHandler(fileHandler)

consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logging.Formatter("%(levelname)-7s ; %(message)s"))
consoleHandler.setLevel(logging.INFO)
logger.addHandler(consoleHandler)

# Model

In [4]:
dbFilename="Wärmenetz-Planungsbeispiel"

# Read Model and Results

In [5]:
m=dxAndMxHelperFcts.readDxAndMx(dbFile=os.path.join(pt3s_path+'/Examples/'+dbFilename+'.db3')
                                ,preventPklDump=True
                               )

INFO    ; Dx.__init__: dbFile (abspath): c:\users\jablonski\3s\pt3s\Examples\Wärmenetz-Planungsbeispiel.db3 exists readable ...
INFO    ; Mx.setResultsToMxsFile: Mxs: ..\Examples\WDWärmenetz-Planungsbeispiel\B1\V0\BZ1\M-1-0-1.1.MXS reading ...
INFO    ; dxWithMx.__init__: Wärmenetz-Planungsbeispiel: processing dx and mx ...


# Interactive Map

In [6]:
gdf_ROHR = m.gdf_ROHR.dropna(subset='geometry')
gdf_FWVB = m.gdf_FWVB.dropna(subset='geometry')

In [7]:
minRadius=2
maxRadius=10*minRadius
facRadius=1/10.

minWidthinPixel=1
maxWidthinPixel=3*minWidthinPixel

facWidthinPixel1DN=1/200
facWidthinPixelQMAVAbs=1/10

In [8]:
dfLonLat=gdf_FWVB.to_crs('EPSG:4326').geometry.get_coordinates()

x_mean=dfLonLat['x'].mean()
y_mean=dfLonLat['y'].mean()

In [9]:
map=folium.Map(location=(y_mean,x_mean),titles='CartoDB Positron',zoom_start=16)

gdf_FWVB.loc[:,['geometry','W']].explore(column='W'
                                         ,cmap='autumn_r'
                                         ,legend=False
                                         ,vmin=gdf_FWVB['W'].quantile(.025),vmax=gdf_FWVB['W'].quantile(.975)
                                         ,style_kwds={'style_function':lambda x: {'radius':min(max(x['properties']['W']*facRadius,minRadius),maxRadius) }}
                                         ,name='W'
                                         ,m=map
                                        )

gdf_ROHR[gdf_ROHR['KVR'].isin([1.,None])].loc[:,['geometry', 'DI']].explore(column='DI'
                                        ,cmap='gray'
                                        ,legend=True
                                        ,vmin=gdf_ROHR['DI'].quantile(.2),vmax=1.5*gdf_ROHR['DI'].quantile(1)
                                        ,style_kwds={'style_function':lambda x: {'radius':min(max(x['properties']['DI']*facWidthinPixel1DN,minWidthinPixel),maxWidthinPixel)}}
                                        ,name='DI'
                                        ,m=map
                                       )

gdf_ROHR[gdf_ROHR['KVR'].isin([1.,None])].loc[:,['geometry', 'QMAVAbs']].explore(column='QMAVAbs'
                                        ,cmap='cool'
                                        ,legend=True
                                        ,vmin=gdf_ROHR['QMAVAbs'].quantile(.2),vmax=gdf_ROHR['QMAVAbs'].quantile(.80)
                                        ,style_kwds={'style_function':lambda x: {'weight':min(max(x['properties']['QMAVAbs']*facWidthinPixelQMAVAbs,minWidthinPixel),maxWidthinPixel)}}
                                        ,name='QMAVAbs'
                                        ,m=map
                                        )

dummy=folium.LayerControl().add_to(map)

In [10]:
map

# Printable Output

In [11]:
img_data = map._to_png(5)
img = Image.open(io.BytesIO(img_data))

In [12]:
img.save('Example1_Output.png')

In [13]:
img.save('Example1_Output.pdf')